In [1]:
from tensorflow.keras.datasets import mnist
import numpy as np

11490434/11490434 [==============================] - 1s 0us/step


>##### MNIST에 백색 잡음 픽셀과 0픽셀 추가하기

In [6]:
(train_images, train_labels), _ = mnist.load_data()                     ## data_load
train_images = train_images.reshape(-1, 28*28)                          ## N sample x height x width => N sample x (height*width) 차원 변경.
print("원래 image size:", train_images.shape)

## Normalization(정규화 진행)
train_images = train_images.astype("float32") / 255

## 노이즈 추가
train_images_with_noise_channels = np.concatenate([train_images, np.random.random((len(train_images), 28*28))], axis=1)
print("Noise 추가 image size:", train_images_with_noise_channels.shape)

## 0 픽셀 추가하기
train_images_with_zeros_channels = np.concatenate([train_images, np.zeros((len(train_images), 28*28))], axis=1)
print("0픽셀 추가 image size:", train_images_with_zeros_channels.shape)

원래 image size: (60000, 784)
Noise 추가 image size: (60000, 1568)
0픽셀 추가 image size: (60000, 1568)


>##### 백색 잡음과 0을 추가한 MNIST 데이터에서 모델 훈련

In [7]:
from tensorflow import keras
from tensorflow.keras import layers

In [8]:
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(512, activation="relu"))
    model.add(keras.layers.Dense(10, activation="softmax"))
    
    model.compile(optimizer="rmsprop",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    
    return model

In [ ]:
model = get_model()
history_noise = model.fit(train_images_with_noise_channels, 
                          train_labels,
                          epochs=10,
                          batch_size=128,
                          validation_split=0.2)

model = get_model()
history_zeros = model.fit(train_images_with_zeros_channels,
                          train_labels,
                          epochs=10)